In [1]:
import pandas as pd
import os
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('Data\\Data.csv',index_col = 0)

D:\Softwares\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
yc = df['ArrDel15']
Xc = df.drop(['ArrDel15','FlightDate','date','place','ArrDelayMinutes','ArrTime'],axis = 1)

In [4]:
yr = df['ArrDelayMinutes']
Xr = df.drop(['ArrDel15','FlightDate','date','place','ArrDelayMinutes','ArrTime'],axis = 1)

In [5]:
from sklearn.model_selection import train_test_split  
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size = 0.20, random_state = 101) 
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size = 0.20, random_state = 101) 

In [6]:
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state = 42)
Xc_train, yc_train = smote.fit_resample(Xc_train, yc_train)

In [7]:
import xgboost 
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(Xc_train, yc_train)

D:\Softwares\Anaconda\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
xgby_pred = xgb.predict(Xc_test)

In [9]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn import metrics

print('Accuracy Score: ',accuracy_score(yc_test, xgby_pred))
cm = metrics.confusion_matrix(yc_test, xgby_pred)
print('Confusion Matrix\n',cm)
target_names=["Class 0","Class 1"]
print('Classification Report\n',classification_report(yc_test, xgby_pred,target_names=target_names))

Accuracy Score:  0.9174056436865747
Confusion Matrix
 [[284515   7784]
 [ 22794  55126]]
Classification Report
               precision    recall  f1-score   support

     Class 0       0.93      0.97      0.95    292299
     Class 1       0.88      0.71      0.78     77920

    accuracy                           0.92    370219
   macro avg       0.90      0.84      0.87    370219
weighted avg       0.92      0.92      0.91    370219



In [10]:
delayed = pd.Series(list(map(bool,xgby_pred)))
delayed

0         False
1         False
2         False
3         False
4         False
          ...  
370214    False
370215    False
370216    False
370217    False
370218    False
Length: 370219, dtype: bool

In [11]:
lst = []
for i in range(len(delayed)):
    if delayed[i] == True:
        lst.append(i)

In [12]:
X_delayed = Xc_test.iloc[lst]
X_delayed

,Quarter,Month,DayofMonth,OriginAirportID,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,...,precipMM,WindGustKmph,pressure,cloudcover,winddirDegree,humidity,DewPointF,tempF,time,WindChillF
Year,,,,,,,,,,,,,,,,,,,,,
2016,4,11,13,12478,14771,14771,925,950.0,25.0,1.0,...,0.0,24,1021,20,245,76,35,43,900,36
2017,2,6,10,13204,11292,11292,2240,2321.0,41.0,1.0,...,1.3,12,1019,45,114,98,73,72,2300,73
2017,4,10,13,11292,13930,13930,1415,1536.0,81.0,1.0,...,0.0,22,1013,0,89,40,36,60,1400,60
2017,2,4,8,11057,11292,11292,2220,2244.0,24.0,1.0,...,0.0,16,1019,0,219,42,33,56,2200,57
2017,2,5,27,14107,10397,10397,1029,1049.0,20.0,1.0,...,0.0,9,1013,0,123,15,30,82,1000,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,1,3,24,11618,12892,12892,1415,1455.0,40.0,1.0,...,1.0,31,1027,86,217,74,35,43,1400,36
2017,1,1,17,11057,12478,12478,1612,1631.0,19.0,1.0,...,0.0,20,1021,96,208,83,56,62,1600,61
2016,4,12,12,12892,12889,12889,1850,1943.0,53.0,1.0,...,0.0,8,1019,100,255,94,59,61,1900,61


In [13]:
y_delayed = yr_test.iloc[lst]
y_delayed

Year
2016     8.0
2017    11.0
2017    78.0
2017     6.0
2017     0.0
        ... 
2017    44.0
2017    31.0
2016    55.0
2016    29.0
2017    42.0
Name: ArrDelayMinutes, Length: 62910, dtype: float64

In [14]:
import xgboost as xg
xgb = xg.XGBRegressor()
xgb.fit(Xr_train,yr_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
y_pred_xgb = xgb.predict(X_delayed)

In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print("mean_squared_error   " , np.sqrt(mean_squared_error(y_delayed, y_pred_xgb)))
print("mean_absolute_error  " , mean_absolute_error(y_delayed, y_pred_xgb))
print("R2score    " , r2_score(y_delayed,y_pred_xgb))

mean_squared_error    17.181550417630074
mean_absolute_error   12.165545609463116
R2score     0.9519089595471744


In [26]:
results = pd.DataFrame()

In [27]:
results['Actual Value'] = y_delayed

In [28]:
results['Predicted Value'] = y_pred_xgb

In [29]:
results

,Actual Value,Predicted Value
Year,,
2016,8.0,19.230303
2017,11.0,29.589062
2017,78.0,79.689102
2017,6.0,16.505245
2017,0.0,13.638557
...,...,...
2017,44.0,29.838533
2017,31.0,14.749866
2016,55.0,54.043320
